In [1]:
import math
from tqdm import tqdm
import torch
import gpytorch
from matplotlib import pyplot as plt
from torch.optim import SGD, Adam

# Make plots inline
%matplotlib inline

In [2]:
from ucimlrepo import fetch_ucirepo 
from math import floor
from torch.utils.data import TensorDataset, DataLoader

# fetch dataset 
real_estate_valuation = fetch_ucirepo(id=477) 
  
# data (as pandas dataframes) 
X_data = real_estate_valuation.data.features 
y_data = real_estate_valuation.data.targets
y = y_data.squeeze()

X = torch.tensor(X_data.values, dtype=torch.float32)
y = torch.tensor(y.values, dtype=torch.float32)

train_n = int(floor(0.8 * len(X)))
train_x = X[:train_n, :].contiguous()
train_y = y[:train_n].contiguous()

test_x = X[train_n:, :].contiguous()
test_y = y[train_n:].contiguous()

# Create TensorDataset and DataLoader for training and test sets
train_dataset = TensorDataset(train_x, train_y)
test_dataset = TensorDataset(test_x, test_y)

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

In [3]:
data_dim = train_x.size(-1)

class LargeFeatureExtractor(torch.nn.Sequential):
    def __init__(self):
        super(LargeFeatureExtractor, self).__init__()
        self.add_module('linear1', torch.nn.Linear(data_dim, 1000))
        self.add_module('relu1', torch.nn.ReLU())
        self.add_module('linear2', torch.nn.Linear(1000, 500))
        self.add_module('relu2', torch.nn.ReLU())
        self.add_module('linear3', torch.nn.Linear(500, 50))
        self.add_module('relu3', torch.nn.ReLU())
        self.add_module('linear4', torch.nn.Linear(50, 2))

feature_extractor = LargeFeatureExtractor()

In [4]:
class GaussianProcessLayer(gpytorch.models.ApproximateGP):
    def __init__(self, num_dim, grid_bounds=(-10., 10.), grid_size=64):
        variational_distribution = gpytorch.variational.CholeskyVariationalDistribution(
            num_inducing_points=grid_size, batch_shape=torch.Size([num_dim])
        )

        variational_strategy = gpytorch.variational.GridInterpolationVariationalStrategy(
                self, grid_size=grid_size, grid_bounds=[grid_bounds],
                variational_distribution=variational_distribution,
            )
        super().__init__(variational_strategy)

        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.RBFKernel(
                lengthscale_prior=gpytorch.priors.SmoothedBoxPrior(
                    math.exp(-1), math.exp(1), sigma=0.1, transform=torch.exp
                )
            )
        )
        self.mean_module = gpytorch.means.ConstantMean()
        self.grid_bounds = grid_bounds

    def forward(self, x):
        mean = self.mean_module(x)
        covar = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean, covar)
    
class DKLModel(gpytorch.Module):
    def __init__(self, feature_extractor, num_dim, grid_bounds=(-10., 10.)):
        super(DKLModel, self).__init__()
        self.feature_extractor = feature_extractor
        self.gp_layer = GaussianProcessLayer(num_dim=num_dim, grid_bounds=grid_bounds)
        self.grid_bounds = grid_bounds
        self.num_dim = num_dim

        self.scale_to_bounds = gpytorch.utils.grid.ScaleToBounds(self.grid_bounds[0], self.grid_bounds[1])

    def forward(self, x):
        features = self.feature_extractor(x)
        #print(features.shape)
        features = self.scale_to_bounds(features)
        features = features.transpose(-1, -2).unsqueeze(-1)
        #print(features.shape)
        res = self.gp_layer(features)
        #print(res)
        return res

# Initialize model and likelihood for regression
model = DKLModel(feature_extractor, num_dim=2)
likelihood = gpytorch.likelihoods.GaussianLikelihood()

original:

In [ ]:
class GaussianProcessLayer(gpytorch.models.ApproximateGP):
    def __init__(self, num_dim, grid_bounds=(-10., 10.), grid_size=64):
        variational_distribution = gpytorch.variational.CholeskyVariationalDistribution(
            num_inducing_points=grid_size, batch_shape=torch.Size([num_dim])
        )

        # Our base variational strategy is a GridInterpolationVariationalStrategy,
        # which places variational inducing points on a Grid
        # We wrap it with a IndependentMultitaskVariationalStrategy so that our output is a vector-valued GP
        variational_strategy = gpytorch.variational.IndependentMultitaskVariationalStrategy(
            gpytorch.variational.GridInterpolationVariationalStrategy(
                self, grid_size=grid_size, grid_bounds=[grid_bounds],
                variational_distribution=variational_distribution,
            ), num_tasks=num_dim,
        )
        super().__init__(variational_strategy)

        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.RBFKernel(
                lengthscale_prior=gpytorch.priors.SmoothedBoxPrior(
                    math.exp(-1), math.exp(1), sigma=0.1, transform=torch.exp
                )
            )
        )
        self.mean_module = gpytorch.means.ConstantMean()
        self.grid_bounds = grid_bounds

    def forward(self, x):
        mean = self.mean_module(x)
        covar = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean, covar)
    
class DKLModel(gpytorch.Module):
    def __init__(self, feature_extractor, num_dim, grid_bounds=(-10., 10.)):
        super(DKLModel, self).__init__()
        self.feature_extractor = feature_extractor
        self.gp_layer = GaussianProcessLayer(num_dim=num_dim, grid_bounds=grid_bounds)
        self.grid_bounds = grid_bounds
        self.num_dim = num_dim

        # This module will scale the NN features so that they're nice values
        self.scale_to_bounds = gpytorch.utils.grid.ScaleToBounds(self.grid_bounds[0], self.grid_bounds[1])

    def forward(self, x):
        features = self.feature_extractor(x)
        features = self.scale_to_bounds(features)
        # This next line makes it so that we learn a GP for each feature
        features = features.transpose(-1, -2).unsqueeze(-1)
        res = self.gp_layer(features)
        return res

model = DKLModel(feature_extractor, num_dim=data_dim)
likelihood = gpytorch.likelihoods.SoftmaxLikelihood(num_features=model.num_dim, num_classes=num_classes)


In [6]:
from torch.optim.lr_scheduler import MultiStepLR
import torch.nn.functional as F
n_epochs = 1
lr = 0.1
optimizer = SGD([
    {'params': model.feature_extractor.parameters(), 'weight_decay': 1e-4},
    {'params': model.gp_layer.hyperparameters(), 'lr': lr * 0.01},
    {'params': model.gp_layer.variational_parameters()},
    {'params': likelihood.parameters()},
], lr=lr, momentum=0.9, nesterov=True, weight_decay=0)
scheduler = MultiStepLR(optimizer, milestones=[0.5 * n_epochs, 0.75 * n_epochs], gamma=0.1)
mll = gpytorch.mlls.VariationalELBO(likelihood, model.gp_layer, num_data=len(train_loader.dataset))


def train(epoch):
    model.train()
    likelihood.train()

    minibatch_iter = tqdm(train_loader, desc=f"(Epoch {epoch}) Minibatch")
    with gpytorch.settings.num_likelihood_samples(8):
        for data, target in minibatch_iter:
            if torch.cuda.is_available():
                data, target = data.cuda(), target.cuda()
            optimizer.zero_grad()
            output = model(data)
            loss = -mll(output, target)
            loss = loss.mean()
            loss.backward()
            optimizer.step()
            minibatch_iter.set_postfix(loss=loss.item())

def test():
    model.eval()
    likelihood.eval()

    total_loss = 0.0
    num_samples = 0
    with torch.no_grad(), gpytorch.settings.num_likelihood_samples(16):
        for data, target in test_loader:
            if torch.cuda.is_available():
                data, target = data.cuda(), target.cuda()
            output = likelihood(model(data))  # This gives us 16 samples from the predictive distribution
            mean = output.mean
            print(mean)
            total_loss += F.l1_loss(mean, target, reduction='sum').item()
            num_samples += target.size(0)
    mae = total_loss / num_samples
    print('Test set: Mean Absolute Error: {:.4f}'.format(mae))

for epoch in range(1, n_epochs + 1):
    with gpytorch.settings.use_toeplitz(False):
        train(epoch)
        test()
    scheduler.step()
    state_dict = model.state_dict()
    likelihood_state_dict = likelihood.state_dict()
    torch.save({'model': state_dict, 'likelihood': likelihood_state_dict}, 'dkl_checkpoint.dat')

(Epoch 1) Minibatch: 100%|██████████| 2/2 [00:00<00:00, 54.06it/s, loss=5.45]

tensor([[ 1.2285,  1.5363,  1.5399,  1.5280,  1.5391,  1.4911,  1.5365,  1.5395,
          1.5391,  1.5394,  1.5340,  1.5399,  1.5370,  1.3927,  1.5397,  1.4998,
          1.2285,  1.5395,  1.5399,  1.5378,  1.4706,  1.4313,  1.4729,  1.5163,
          1.5388,  1.5397,  1.5391,  1.5397,  1.4534,  1.5399,  1.5394,  1.5292,
          1.5399,  1.5363,  1.4648,  1.4949,  1.4934,  1.5390,  1.4703,  1.5394,
          1.5392,  1.5398,  1.5394,  1.5387,  1.5237,  1.4996,  1.5399,  1.5367,
          1.5393,  1.5394,  1.5398,  1.3451,  1.5377,  1.2813,  1.5399,  1.5397,
          1.4760,  1.4654,  1.5399,  1.5388,  1.4866,  1.5383,  1.5292,  1.2727,
          1.5376,  1.5305,  1.5234,  1.4996,  1.5398,  1.5379,  1.4710,  1.5397,
          1.5398,  1.5394,  1.5398,  1.5389,  1.4534,  1.4710,  1.2794,  1.5399,
          1.5387,  1.5399,  1.5399],
        [ 0.1517,  0.0797,  0.2066,  0.0328,  0.1422,  0.0050,  0.0813,  0.1581,
          0.1436,  0.1552,  0.0715,  0.2146,  0.0856,  0.1339,  0.1776, 


C:\Users\12lui\AppData\Local\Temp\ipykernel_12584\190988355.py:45: UserWarning: Using a target size (torch.Size([83])) that is different to the input size (torch.Size([2, 83])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  total_loss += F.l1_loss(mean, target, reduction='sum').item()
